In [ ]:
!pip install -U openai-whisper
!pip install setuptools-rust
!pip install pytube
!apt -y install ffmpeg lame

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.9/792.9 kB 10.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 57.8 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=796907 sha256=57e5ed042fc42051efed5cff10e4bd63e37af489cd1ea1729c68ba2dc62df4b9
  Stored in directory: /root/.cache/pip/wheels/b2/13/5f/fe8245f6dc59df505879da4b2129932e342f02a80e6b87f27d
Successfully built openai-whisper
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
Suggested packages:
  lame-doc
The following NEW packages will be installed:
  lame
0 upgraded, 1 newly installed, 0 to remove and 16 not upgraded.
Need

In [ ]:
!pip install pyngrok==4.1.1
!pip install Flask eventlet flask-socketio
!pip install flask_ngrok
!pip install flask_cors

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=564387c7c9461fc266a2651bdf271a4311327d5a8a7ea1a78b620ad78283e7c2
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.3 MB/s eta 0:00:00


In [ ]:
from pyngrok import ngrok
!ngrok authtoken "2Ogwlhxbd6wfxfvpGu0rx4JCS6U_5Gqo9WE95gtG2SfetXMxu"

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from pytube import YouTube

video_url = "https://www.youtube.com/watch?v=VmFFlK03WHc&pp=ygUQ7ZWc6rWt7IKsIOqwleydmA%3D%3D"

yt = YouTube(video_url)
stream = yt.streams.filter(only_audio=True).first()
output_directory = '/content/'
output_filename = 'file.mp3'

# MP3 파일로 저장합니다.
stream.download(output_path=output_directory, filename=output_filename)

'/content/file.mp3'

In [ ]:
#%%writefile app.py

from pyngrok import ngrok
from flask_cors import CORS
from flask_socketio import SocketIO
from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify, request, send_file

import sys
import tqdm
import threading
from typing import Union

import os
import whisper
import zipfile
from whisper.utils import get_writer
from pytube import YouTube
from pathlib import Path


app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'
socketio = SocketIO(app)
run_with_ngrok(app)
CORS(app)


@socketio.on('message')
def handle_message(message):
    print('Received message:', message)
    socketio.emit('response', 'Server received your message: ' + message)


class ProgressListener:
    def on_progress(self, current: Union[int, float], total: Union[int, float]):
        self.total = total

    def on_finished(self):
        pass

class _CustomProgressBar(tqdm.tqdm):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._current = self.n  # Set the initial value

    def update(self, n):
        super().update(n)
        # Because the progress bar might be disabled, we need to manually update the progress
        self._current += n

        # Inform listeners
        listeners = _get_thread_local_listeners()

        for listener in listeners:
            listener.on_progress(self._current, self.total)

_thread_local = threading.local()

def _get_thread_local_listeners():
    if not hasattr(_thread_local, 'listeners'):
        _thread_local.listeners = []
    return _thread_local.listeners

_hooked = False

def init_progress_hook():
    global _hooked

    if _hooked:
        return

    # Inject into tqdm.tqdm of Whisper, so we can see progress
    import whisper.transcribe
    transcribe_module = sys.modules['whisper.transcribe']
    transcribe_module.tqdm.tqdm = _CustomProgressBar
    _hooked = True

def register_thread_local_progress_listener(progress_listener: ProgressListener):
    # This is a workaround for the fact that the progress bar is not exposed in the API
    init_progress_hook()

    listeners = _get_thread_local_listeners()
    listeners.append(progress_listener)

def unregister_thread_local_progress_listener(progress_listener: ProgressListener):
    listeners = _get_thread_local_listeners()

    if progress_listener in listeners:
        listeners.remove(progress_listener)

class ProgressListenerHandle:
    def __init__(self, listener: ProgressListener):
        self.listener = listener

    def __enter__(self):
        register_thread_local_progress_listener(self.listener)

    def __exit__(self, exc_type, exc_val, exc_tb):
        unregister_thread_local_progress_listener(self.listener)

        if exc_type is None:
            self.listener.on_finished()

def create_progress_listener_handle(progress_listener: ProgressListener):
    return ProgressListenerHandle(progress_listener)

class PrintingProgressListener:
    def on_progress(self, current: Union[int, float], total: Union[int, float]):
      print(f"Progress: {current}/{total}")
      #socket
    def on_finished(self):
      print("Finished")



@app.route('/', methods=['GET'])
def GETA():
  if request.method == 'GET':
    return jsonify({ "test": "abc" })

@app.route('/', methods=['POST'])
def GETB():
    params = request.form
    content = request.files['uploadFile']
    print(params)

    model = whisper.load_model(params['selectedModel'])
    fromlanguage = params['selectedFromLanguage']
    tolanguage = params['selectedToLanguage']


    options = dict(language=language, word_timestamps=True, verbose=False)

    transcribe_options = dict(task="transcribe", **options)
    translate_options = dict(task="translate", **options)

    if fromlanguage != 'en' and tolanguage == 'en':
      with create_progress_listener_handle(PrintingProgressListener()) as listener:
        result = model.transcribe(content, **translate_options)
    elif fromlanguage != 'en' and tolanguage != 'en' and fromlanguage != tolanguage:
      with create_progress_listener_handle(PrintingProgressListener()) as listener:
        result = model.transcribe(content, **translate_options)
    else:
      with create_progress_listener_handle(PrintingProgressListener()) as listener:
        result = model.transcribe(content, **transcribe_options)

    file_path = Path(content)
    txt_path = file_path.with_suffix(".txt")
    with open(txt_path, "w", encoding="utf-8") as txt:
        txt.write(result["text"])

    output_directory = file_path.parent
    srt_writer = get_writer("srt", output_directory)
    srt_writer(result, str(file_path.stem))

    vtt_writer = get_writer("vtt", output_directory)
    vtt_writer(result, str(file_path.stem))

    tsv_writer = get_writer("tsv", output_directory)
    tsv_writer(result, str(file_path.stem))

    file_paths = ['/content/file.srt','/content/file.tsv','/content/file.txt','/content/file.vtt']

    zip_filename = 'downloaded_files.zip'
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file_path in file_paths:
            zipf.write(file_path, os.path.basename(file_path))

    return send_file('/content/file.srt', as_attachment=True)

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://dfb0-34-80-21-16.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [11/Sep/2023 03:58:15] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Sep/2023 03:58:15] "GET /favicon.ico HTTP/1.1" 404 -


ImmutableMultiDict([('selectedLanguage', 'ko'), ('selectedModel', 'medium'), ('selectedVad', 'silero-vad'), ('youtube', 'https://www.youtube.com/watch?v=VmFFlK03WHc&pp=ygUQ7ZWc6rWt7IKsIOqwleydmA%3D%3D'), ('uploadFile', 'None'), ('record', 'None'), ('task', 'false'), ('mergeWindows', '5'), ('maxMergeSize', '30'), ('isTimestamps', 'true'), ('isHighlight', 'false')])


...

Progress: 86400/36371
Progress: 89022/36371
Progress: 91644/36371


 92%|█████████▏| 33410/36371 [01:44<00:09, 323.24frames/s]

Progress: 94506/36371
Progress: 97368/36371
Progress: 100230/36371


100%|██████████| 36371/36371 [01:47<00:00, 339.71frames/s]
INFO:werkzeug:127.0.0.1 - - [11/Sep/2023 04:00:34] "POST / HTTP/1.1" 200 -


Progress: 103191/36371
Progress: 106152/36371
Progress: 109113/36371
Finished
